In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# Config
# ---------------------------
np.random.seed(42)

OUT_DATA_DIR = "data"
OUT_IMG_DIR = "exports"
os.makedirs(OUT_DATA_DIR, exist_ok=True)
os.makedirs(OUT_IMG_DIR, exist_ok=True)

months = pd.date_range("2024-01-01", periods=48, freq="MS")  # 24 months actuals + 24 months forecast window feel
n = len(months)

# ---------------------------
# Synthetic corporate finance drivers
# ---------------------------
base_revenue = 10_000_000  # starting monthly revenue
growth = 0.006             # ~0.6% monthly growth (approx)
seasonality = 1 + 0.08 * np.sin(np.linspace(0, 6*np.pi, n))  # smooth seasonality
noise = np.random.normal(0, 0.03, n)                        # 3% noise

revenue = base_revenue * (1 + growth) ** np.arange(n)
revenue = revenue * seasonality * (1 + noise)
revenue = np.maximum(revenue, 0)

# Cost structure (as % of revenue with drift)
cogs_rate = 0.52 + np.random.normal(0, 0.01, n) + np.linspace(0.0, 0.02, n)  # slow inflation drift
opex_rate = 0.28 + np.random.normal(0, 0.008, n)                              # relatively stable

cogs = revenue * cogs_rate
opex = revenue * opex_rate

# EBITDA and margin
ebitda = revenue - cogs - opex
margin = ebitda / revenue

# Cash flow proxy (simplified)
cashflow = ebitda - 0.05 * revenue + np.random.normal(0, 0.01, n) * revenue

# Category split for expenses (for variance dashboard)
opex_categories = ["Payroll", "Marketing", "Cloud/IT", "G&A"]
opex_split = np.array([0.45, 0.20, 0.18, 0.17])
opex_cat = {cat: opex * opex_split[i] * (1 + np.random.normal(0, 0.03, n)) for i, cat in enumerate(opex_categories)}

# ---------------------------
# Budget (plan): smoother, slightly optimistic revenue, tighter costs
# ---------------------------
budget_revenue = (base_revenue * (1 + growth) ** np.arange(n)) * (1 + 0.03) * seasonality
budget_cogs = budget_revenue * (0.50 + np.linspace(0.0, 0.015, n))
budget_opex = budget_revenue * 0.27

# ---------------------------
# Build tables
# ---------------------------
df = pd.DataFrame({
    "Month": months,
    "Revenue": revenue,
    "COGS": cogs,
    "Opex": opex,
    "EBITDA": ebitda,
    "EBITDA_Margin": margin,
    "CashFlowProxy": cashflow,
})

for cat in opex_categories:
    df[f"Opex_{cat}"] = opex_cat[cat]

df_budget = pd.DataFrame({
    "Month": months,
    "BudgetRevenue": budget_revenue,
    "BudgetCOGS": budget_cogs,
    "BudgetOpex": budget_opex,
})

# Variances
df_var = df.merge(df_budget, on="Month", how="left")
df_var["Rev_Var_$"] = df_var["Revenue"] - df_var["BudgetRevenue"]
df_var["Rev_Var_%"] = df_var["Rev_Var_$"] / df_var["BudgetRevenue"]
df_var["Opex_Var_$"] = df_var["Opex"] - df_var["BudgetOpex"]
df_var["Opex_Var_%"] = df_var["Opex_Var_$"] / df_var["BudgetOpex"]
df_var["Margin_Var_pp"] = (df_var["EBITDA_Margin"] - ((df_var["BudgetRevenue"]-df_var["BudgetCOGS"]-df_var["BudgetOpex"]) / df_var["BudgetRevenue"])) * 100

# ---------------------------
# Scenarios (simple but credible)
# ---------------------------
# Base forecast assumptions already embedded in budget; we'll build scenarios off budget.
downside_revenue = budget_revenue * 0.92  # 8% revenue downside
inflation_cogs = budget_cogs * 1.07      # 7% cogs inflation shock
inflation_opex = budget_opex * 1.03      # 3% opex inflation shock

base_ebitda = budget_revenue - budget_cogs - budget_opex
downside_ebitda = downside_revenue - budget_cogs - budget_opex
inflation_ebitda = budget_revenue - inflation_cogs - inflation_opex

df_scen = pd.DataFrame({
    "Month": months,
    "Base_EBITDA": base_ebitda,
    "Downside_EBITDA": downside_ebitda,
    "Inflation_EBITDA": inflation_ebitda,
    "Base_Margin": base_ebitda / budget_revenue,
    "Downside_Margin": downside_ebitda / downside_revenue,
    "Inflation_Margin": inflation_ebitda / budget_revenue,
})

# ---------------------------
# Save data
# ---------------------------
df.to_csv(os.path.join(OUT_DATA_DIR, "fpa_monthly.csv"), index=False)
df_budget.to_csv(os.path.join(OUT_DATA_DIR, "fpa_budget.csv"), index=False)
df_scen.to_csv(os.path.join(OUT_DATA_DIR, "fpa_scenarios.csv"), index=False)

# ---------------------------
# Helper: formatting
# ---------------------------
def fmt_millions(x):
    return f"${x/1_000_000:.1f}M"

# ---------------------------
# 1) KPI strip (as a simple figure)
# ---------------------------
last = df_var.iloc[-1]
kpis = {
    "Revenue vs Plan": f"{(last['Revenue']/last['BudgetRevenue'] - 1)*100:.1f}%",
    "Expense Variance": f"{(last['Opex']/last['BudgetOpex'] - 1)*100:.1f}%",
    "EBITDA Margin": f"{last['EBITDA_Margin']*100:.1f}%",
    "Margin Δ (pp)": f"{last['Margin_Var_pp']:.1f}",
}

# Risk flag logic (simple, interview-safe)
risk_flag = "Green"
if last["Opex_Var_%"] > 0.06 or last["EBITDA_Margin"] < 0.12:
    risk_flag = "Amber"
if last["Opex_Var_%"] > 0.10 or last["EBITDA_Margin"] < 0.09:
    risk_flag = "Red"

plt.figure(figsize=(12, 2.2))
plt.axis("off")
txt = " | ".join([f"{k}: {v}" for k, v in kpis.items()] + [f"Risk Flag: {risk_flag}"])
plt.text(0.01, 0.5, txt, fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "kpi_strip.png"), dpi=200, bbox_inches="tight")
plt.close()

# ---------------------------
# 2) Revenue Actual vs Budget
# ---------------------------
plt.figure(figsize=(12, 5))
plt.plot(df_var["Month"], df_var["Revenue"], label="Actual Revenue")
plt.plot(df_var["Month"], df_var["BudgetRevenue"], label="Budget Revenue")
plt.title("Revenue: Actual vs Budget (24+ Month View)")
plt.ylabel("USD")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "revenue_actual_vs_budget.png"), dpi=200)
plt.close()

# ---------------------------
# 3) Expense variance by category (latest month)
# ---------------------------
latest_month = df_var["Month"].iloc[-1]
actual_cats = [df.loc[df["Month"] == latest_month, f"Opex_{c}"].values[0] for c in opex_categories]
budget_total = df_budget.loc[df_budget["Month"] == latest_month, "BudgetOpex"].values[0]
# Allocate budget by same split for display
budget_cats = list(budget_total * opex_split)

var_cats = np.array(actual_cats) - np.array(budget_cats)

plt.figure(figsize=(10, 5))
plt.bar(opex_categories, var_cats)
plt.title(f"Opex Variance by Category (Latest Month: {latest_month.strftime('%b %Y')})")
plt.ylabel("Variance ($)")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "expense_variance_by_category.png"), dpi=200)
plt.close()

# ---------------------------
# 4) Margin trend
# ---------------------------
plt.figure(figsize=(12, 5))
plt.plot(df_var["Month"], df_var["EBITDA_Margin"]*100)
plt.title("EBITDA Margin Trend")
plt.ylabel("Margin (%)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "margin_trend.png"), dpi=200)
plt.close()

# ---------------------------
# 5) Scenario EBITDA comparison
# ---------------------------
plt.figure(figsize=(12, 5))
plt.plot(df_scen["Month"], df_scen["Base_EBITDA"], label="Base")
plt.plot(df_scen["Month"], df_scen["Downside_EBITDA"], label="Revenue Downside")
plt.plot(df_scen["Month"], df_scen["Inflation_EBITDA"], label="Cost Inflation")
plt.title("Scenario Comparison: EBITDA Impact")
plt.ylabel("USD")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "scenario_ebitda_comparison.png"), dpi=200)
plt.close()

# ---------------------------
# 6) Variance heatmap (Revenue variance % vs time)
# ---------------------------
# Simple heatmap using a single row matrix for clean embedding
heat = df_var["Rev_Var_%"].values.reshape(1, -1)

plt.figure(figsize=(12, 2.5))
plt.imshow(heat, aspect="auto")
plt.yticks([])
plt.xticks(ticks=np.arange(0, n, 3), labels=[m.strftime("%b %y") for m in df_var["Month"][::3]], rotation=45, ha="right")
plt.title("Revenue Variance % Heatmap (Budget vs Actual)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_IMG_DIR, "variance_heatmap.png"), dpi=200)
plt.close()

print("Done. Data saved to /data and images saved to /exports.")


Done. Data saved to /data and images saved to /exports.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')